In [2]:
# default_exp cli

# CLI

This notebook includes helper functions and processes used in the development of the CLI for ETS-Watch

<br>

### Imports

In [2]:
#exports
from etswatch import prices
from etswatch.eutl import accounts

import os
import typer

In [26]:
import dotenv

<br>

### Loading Environment Variables

In [24]:
use_dotenv = True

if use_dotenv == True:
    assert dotenv.load_dotenv(), 'environment variables could not be loaded'

<br>

### Initialising CLI 

In [25]:
#exports
app = typer.Typer()

<br>

### Download Commands

We'll start by creating a function for downloading the market prices data

In [5]:
#exports
@app.command()
def download_mkt_prices(data_dir='data', print_dataset_head=True):
    api_key = os.getenv('QUANDL_API_KEY')
    df_ets = prices.get_ets_mkt_data(api_key)
    
    df_ets.to_csv(f'{data_dir}/market_prices.csv')
    
    if print_dataset_head == True:
        print(f'Output Tail (n=3):\n\n{df_ets.tail(3)}')
    
    return

In [6]:
download_mkt_prices(data_dir='../data')

c:\users\ayrto\desktop\side projects\ets-watch\etswatch\prices.py:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.lower().str.replace('.', '').str.replace(' ', '_')


Output Tail (n=3):

             open   high    low  settle  change   wave  volume  \
datetime                                                         
2021-04-14  43.87  43.88  43.47   43.73   -0.03  43.75    28.0   
2021-04-15  43.84  44.32  43.84   44.08    0.35  44.22     5.0   
2021-04-16  44.53  44.53  44.53   44.33    0.25  44.53     1.0   

            prev_day_open_interest  efp_volume  efs_volume  block_volume  \
datetime                                                                   
2021-04-14                  4865.0         NaN         NaN        2290.0   
2021-04-15                  2881.0         NaN         NaN         598.0   
2021-04-16                  2483.0         NaN         NaN           NaN   

            close  
datetime           
2021-04-14  43.84  
2021-04-15  44.19  
2021-04-16  44.78  


<br>

Next we'll download the aircraft data

In [13]:
#exports
@app.command()
def download_aircraft_accounts(data_dir='data', redownload=True, print_dataset_heads=True):
    df_search = accounts.get_search_df(data_dir=f'{data_dir}/..', redownload=redownload)   
    
    aircraft_dfs = accounts.get_aircraft_dfs(
        df_search, 
        data_dir=data_dir, 
        redownload=redownload
    )
    
    if print_dataset_heads == True:
        for dataset_name, df in aircraft_dfs.items():
            print(f'{dataset_name} (head, n=3):\n\n{df.head(3)}\n\n\n')
    
    return

In [14]:
download_aircraft_accounts(data_dir='../data/aircraft', redownload=False)

aircraft (head, n=3):

   account_id  aircraft_operator_id Unnamed: 2             monitoring_plan_id  \
0       90574                200103      27702  BMLFUW-UW.1.3.2/0354-V/4/2009   
1       90727                200108        194  BMFLUW-UW.1.3.2/0084-V/4/2010   
2       90728                200109      36057         UW.1.3.2/0085-V/4/2011   

  monitoring_plan_start_date  monitoring_plan_expiration_Date  \
0                 2010-01-01                              NaN   
1                 2010-01-01                              NaN   
2                 2010-01-01                              NaN   

   subsidiary_undertaking_name  parent_undertaking_name  EPRTR_id call_sign  \
0                          NaN                      NaN       NaN       JAG   
1                          NaN                      NaN       NaN       NaN   
2                          NaN                      NaN       NaN       NaN   

   initial_emissions_year  first_address_line  second_address_line  postco

<br>

We'll then do the same for the installation accounts

In [15]:
#exports
@app.command()
def download_installation_accounts(data_dir='data', redownload=True, print_dataset_heads=True):
    df_search = accounts.get_search_df(data_dir=f'{data_dir}/..', redownload=redownload)   
    
    installation_dfs = accounts.get_installation_dfs(
        df_search, 
        data_dir=data_dir, 
        redownload=redownload
    )
    
    if print_dataset_heads == True:
        for dataset_name, df in installation_dfs.items():
            print(f'{dataset_name} (head, n=3):\n\n{df.head(3)}\n\n\n')
    
    return

In [16]:
download_installation_accounts(data_dir='../data/installations', redownload=False)

allocated_allowances (head, n=3):

   account_id      2005      2006      2007      2008      2009      2010  \
0      100000  269940.0  269941.0  269941.0  153922.0  153922.0  153922.0   
1      100001  146438.0  146438.0  146438.0  131200.0  131197.0  131197.0   
2      100002      10.0      11.0       0.0       NaN       NaN       NaN   

       2011      2012   2013  ... 2021 2022 2023 2024 2025 2026 2027  2028  \
0  153922.0  153922.0   3167  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   
1  131197.0  131197.0  16892  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   
2       NaN       NaN    NaN  ...  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   

   2029  2030  
0   NaN   NaN  
1   NaN   NaN  
2   NaN   NaN  

[3 rows x 27 columns]



compliance_code (head, n=3):

   account_id 2005 2006 2007 2008 2009 2010 2011 2012 2013  ... 2021 2022  \
0      100000    A    A    A    A    A    A    A    A    A  ...  NaN  NaN   
1      100001    A    A    A    A    A    A    A    A    A  ...  NaN 

<br>

Lastly we'll create a function for downloading all of the accounts data in one step (including the accounts search data)

In [20]:
#exports
@app.command()
def download_all_accounts_data(data_dir='data', search=True, installations=True, aircraft=True):
    all_dfs = accounts.retrieve_all_data(data_dir=data_dir, redownload_search=search, redownload_installations=installations, redownload_aircraft=aircraft)   
    
    return

In [22]:
download_all_accounts_data(data_dir='../data', search=False, installations=False, aircraft=False)

<br>

### Context Handling

Finally we need to ensure the CLI app is available when the module is loaded.

N.b. we've included the condition `'__file__' in globals()` to make sure this isn't when inside the notebook

In [18]:
#exports
if __name__ == '__main__' and '__file__' in globals():
    app()

In [1]:
#hide
from nbdev.export import notebook2script
notebook2script('05-cli.ipynb')

Converted 04-cli.ipynb.
